In [23]:
import numpy as np
from pulp import LpProblem, LpVariable, lpSum, LpMaximize

# The original generate_distances function remains unchanged
def generate_distances(num_cities, seed):
    np.random.seed(seed)  # Set the seed for reproducibility
    # Create a random distance matrix (lower triangle only for symmetry)
    distances = np.random.randint(1, 1000, size=(num_cities, num_cities))
    # Make the matrix symmetric
    distances = np.triu(distances, 1) + np.triu(distances, 1).T
    # Set the diagonal to 0 (distance from a city to itself)
    np.fill_diagonal(distances, 0)
    return distances

# Function to calculate travel times from the distance matrix
def calculate_travel_times(distance_matrix, speed):
    return distance_matrix / speed

# Function to setup the optimization problem
def setup_optimization_model(cities, num_cities, delivery_values, travel_times, time_windows):
    model = LpProblem("TSP_Single_Objective", LpMaximize)

    # Decision Variables
    visit = LpVariable.dicts("Visit", range(num_cities), cat="Binary")
    travel = LpVariable.dicts("Travel", [(i, j) for i in range(num_cities) for j in range(num_cities)], cat="Binary")
    arrival_time = LpVariable.dicts("ArrivalTime", range(num_cities), lowBound=0, cat="Continuous")

    # Objective Function: Maximize total delivery value
    model += lpSum(delivery_values[i] * visit[i] for i in range(num_cities)), "Total_Delivery_Value"

    # Constraints
    # 1. Start and end at city 0 (index 0)
    model += visit[0] == 1, "Start_at_0"
    model += lpSum(travel[(0, j)] for j in range(1, num_cities)) == 1, "Leave_0_Once"
    model += lpSum(travel[(i, 0)] for i in range(1, num_cities)) == 1, "Return_to_0"

    # 2. Visit between 3 and 5 cities
    model += lpSum(visit[i] for i in range(num_cities)) >= 3, "Minimum_Visits"
    model += lpSum(visit[i] for i in range(num_cities)) <= 5, "Maximum_Visits"

    # 3. Dependency: If city 1 (B) is visited, city 5 (F) must be visited
    model += visit[1] <= visit[5], "Dependency_1_to_5"

    # 4. Mutual Exclusion: Either city 2 (C) or 3 (D) can be visited, not both
    model += visit[2] + visit[3] <= 1, "Mutual_Exclusion_2_3"

    # 5. Mutual Exclusion: Either city 6 (G) or 7 (H) can be visited, not both
    model += visit[6] + visit[7] <= 1, "Mutual_Exclusion_6_7"

    # 6. Time window constraints
    for i in range(num_cities):
        model += arrival_time[i] >= time_windows[i][0] * visit[i], f"TimeWindowStart_{i}"
        model += arrival_time[i] <= time_windows[i][1] * visit[i], f"TimeWindowEnd_{i}"

    # 7. Sequential travel constraints: Arrival times must respect travel durations
    M = 1e6  # A large constant for linearization
    for i in range(num_cities):
        for j in range(num_cities):
            if i != j:
                model += (
                    arrival_time[j]
                    >= arrival_time[i] + travel_times[i, j] - M * (1 - travel[(i, j)])
                ), f"TravelTime_{i}_to_{j}"

    return model, visit, arrival_time

# Function to print results
def print_results(model, cities, visit, arrival_time):
    model.solve()

    visited_cities = [cities[i] for i in range(len(cities)) if visit[i].value() == 1]
    optimal_value = model.objective.value()

    print("Visited Cities:", visited_cities)
    print("Optimal Delivery Value:", optimal_value)
    print("Arrival Times:")
    for i in range(len(cities)):
        if visit[i].value() == 1:
            print(f"{cities[i]}: {arrival_time[i].value()}")

# Main execution
if __name__ == "__main__":
    # Define cities, delivery values, and time windows
    cities = ["A", "B", "C", "D", "E", "F", "G", "H"]
    num_cities = len(cities)

    delivery_values = [0, 500, 300, 400, 600, 200, 350, 450]
    time_windows = [(0, 24), (9, 12), (10, 14), (11, 13), (13, 17), (8.5, 10.5), (12, 16), (14, 18)]

    # Generate distance matrix and calculate travel times
    seed = 5286
    distance_matrix = generate_distances(num_cities, seed)
    speed = 30  # km/h
    travel_times = calculate_travel_times(distance_matrix, speed)

    # Set up the optimization model
    model, visit, arrival_time = setup_optimization_model(cities, num_cities, delivery_values, travel_times, time_windows)

    # Display the results
    print_results(model, cities, visit, arrival_time)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/b9/rs9ddddj2qb9xx5yj7h2b03w0000gn/T/236493e067204fdcbd0a8e215d8e5000-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/b9/rs9ddddj2qb9xx5yj7h2b03w0000gn/T/236493e067204fdcbd0a8e215d8e5000-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 85 COLUMNS
At line 457 RHS
At line 538 BOUNDS
At line 603 ENDATA
Problem MODEL has 80 rows, 72 columns and 236 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1800 - 0.00 seconds
Cgl0003I 7 fixed, 0 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 22 rows, 17 columns (9 integer (9 of which binary)) and 53 elements
Cbc0006I The LP relaxation is infeasible or too expensive

In [24]:
import numpy as np
from pulp import LpProblem, LpVariable, lpSum, LpMaximize

# The original generate_distances function remains unchanged
def generate_distances(num_cities, seed):
    np.random.seed(seed)  # Set the seed for reproducibility
    # Create a random distance matrix (lower triangle only for symmetry)
    distances = np.random.randint(1, 1000, size=(num_cities, num_cities))
    # Make the matrix symmetric
    distances = np.triu(distances, 1) + np.triu(distances, 1).T
    # Set the diagonal to 0 (distance from a city to itself)
    np.fill_diagonal(distances, 0)
    return distances

# Function to calculate travel times from the distance matrix
def calculate_travel_times(distance_matrix, speed):
    return distance_matrix / speed

# Function to setup the optimization model with two objectives
def setup_optimization_model(cities, num_cities, delivery_values, travel_times, time_windows, weight_profit=0.5, weight_distance=0.5):
    model = LpProblem("TSP_Multi_Objective", LpMaximize)

    # Decision Variables
    visit = LpVariable.dicts("Visit", range(num_cities), cat="Binary")
    travel = LpVariable.dicts("Travel", [(i, j) for i in range(num_cities) for j in range(num_cities)], cat="Binary")
    arrival_time = LpVariable.dicts("ArrivalTime", range(num_cities), lowBound=0, cat="Continuous")

    # First Objective Function: Maximize total delivery value (profit)
    profit_objective = lpSum(delivery_values[i] * visit[i] for i in range(num_cities))

    # Second Objective Function: Minimize total distance traveled
    distance_objective = lpSum(travel_times[i, j] * travel[(i, j)] for i in range(num_cities) for j in range(num_cities) if i != j)

    # Combine both objectives using weighted sum
    model += weight_profit * profit_objective - weight_distance * distance_objective, "Total_Objective"

    # Constraints
    # 1. Start and end at city 0 (index 0)
    model += visit[0] == 1, "Start_at_0"
    model += lpSum(travel[(0, j)] for j in range(1, num_cities)) == 1, "Leave_0_Once"
    model += lpSum(travel[(i, 0)] for i in range(1, num_cities)) == 1, "Return_to_0"

    # 2. Visit between 3 and 5 cities
    model += lpSum(visit[i] for i in range(num_cities)) >= 3, "Minimum_Visits"
    model += lpSum(visit[i] for i in range(num_cities)) <= 5, "Maximum_Visits"

    # 3. Dependency: If city 1 (B) is visited, city 5 (F) must be visited
    model += visit[1] <= visit[5], "Dependency_1_to_5"

    # 4. Mutual Exclusion: Either city 2 (C) or 3 (D) can be visited, not both
    model += visit[2] + visit[3] <= 1, "Mutual_Exclusion_2_3"

    # 5. Mutual Exclusion: Either city 6 (G) or 7 (H) can be visited, not both
    model += visit[6] + visit[7] <= 1, "Mutual_Exclusion_6_7"

    # 6. Time window constraints
    for i in range(num_cities):
        model += arrival_time[i] >= time_windows[i][0] * visit[i], f"TimeWindowStart_{i}"
        model += arrival_time[i] <= time_windows[i][1] * visit[i], f"TimeWindowEnd_{i}"

    # 7. Sequential travel constraints: Arrival times must respect travel durations
    M = 1e6  # A large constant for linearization
    for i in range(num_cities):
        for j in range(num_cities):
            if i != j:
                model += (
                    arrival_time[j]
                    >= arrival_time[i] + travel_times[i, j] - M * (1 - travel[(i, j)])
                ), f"TravelTime_{i}_to_{j}"

    return model, visit, arrival_time, profit_objective, distance_objective

# Function to print results
def print_results(model, cities, visit, arrival_time):
    model.solve()

    visited_cities = [cities[i] for i in range(len(cities)) if visit[i].value() == 1]
    optimal_value = model.objective.value()

    print("Visited Cities:", visited_cities)
    print("Optimal Objective Value (Profit - Distance):", optimal_value)
    print("Arrival Times:")
    for i in range(len(cities)):
        if visit[i].value() == 1:
            print(f"{cities[i]}: {arrival_time[i].value()}")

# Main execution
if __name__ == "__main__":
    # Define cities, delivery values, and time windows
    cities = ["A", "B", "C", "D", "E", "F", "G", "H"]
    num_cities = len(cities)

    delivery_values = [0, 500, 300, 400, 600, 200, 350, 450]
    time_windows = [(0, 24), (9, 12), (10, 14), (11, 13), (13, 17), (8.5, 10.5), (12, 16), (14, 18)]

    # Generate distance matrix and calculate travel times
    seed = 5286
    distance_matrix = generate_distances(num_cities, seed)
    speed = 30  # km/h
    travel_times = calculate_travel_times(distance_matrix, speed)

    # Set up the optimization model with weights for objectives
    weight_profit = 0.5  # Weight for profit (maximize delivery values)
    weight_distance = 0.5  # Weight for distance (minimize total distance)
    
    model, visit, arrival_time, profit_objective, distance_objective = setup_optimization_model(
        cities, num_cities, delivery_values, travel_times, time_windows, 
        weight_profit, weight_distance)

    # Display the results
    print_results(model, cities, visit, arrival_time)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/b9/rs9ddddj2qb9xx5yj7h2b03w0000gn/T/315361f0c5414c62ae605b2c6eed3127-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/b9/rs9ddddj2qb9xx5yj7h2b03w0000gn/T/315361f0c5414c62ae605b2c6eed3127-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 85 COLUMNS
At line 513 RHS
At line 594 BOUNDS
At line 659 ENDATA
Problem MODEL has 80 rows, 72 columns and 236 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 888.8 - 0.00 seconds
Cgl0003I 9 fixed, 0 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 17 rows, 14 columns (7 integer (7 of which binary)) and 41 elements
Cbc0038I Initial state - 3 integers unsatisfied sum - 1.

# Logistic Reg.

In [30]:
import numpy as np
from random import randint, random, shuffle

# Assuming the distance and time window functions are already defined from the previous question

# Objective functions (Total Delivery Value and Total Travel Time)
def objective_1(visit, delivery_values):
    return sum(delivery_values[i] * visit[i] for i in range(len(visit)))

def objective_2(visit, travel_times, num_cities):
    total_time = 0
    for i in range(num_cities):
        for j in range(num_cities):
            if i != j and visit[i] == 1 and visit[j] == 1:
                total_time += travel_times[i, j] * visit[i] * visit[j]  # Travel only between visited cities
    return total_time

# Non-dominated sorting and crowding distance functions (from NSGA-II)
def non_dominated_sorting(population, delivery_values, travel_times):
    fronts = []
    for i, p in enumerate(population):
        p['dominated_by'] = set()
        p['dominates'] = set()
        p['rank'] = 0
        p['crowding_distance'] = 0

    # Find domination relationships
    for i in range(len(population)):
        for j in range(i+1, len(population)):
            obj_1_i = objective_1(population[i]['visit'], delivery_values)
            obj_2_i = objective_2(population[i]['visit'], travel_times, len(delivery_values))
            obj_1_j = objective_1(population[j]['visit'], delivery_values)
            obj_2_j = objective_2(population[j]['visit'], travel_times, len(delivery_values))

            if (obj_1_i >= obj_1_j and obj_2_i >= obj_2_j):
                population[i]['dominates'].add(j)
                population[j]['dominated_by'].add(i)
            elif (obj_1_j >= obj_1_i and obj_2_j >= obj_2_i):
                population[j]['dominates'].add(i)
                population[i]['dominated_by'].add(j)

    # Create fronts
    fronts.append([i for i in range(len(population)) if len(population[i]['dominated_by']) == 0])
    # Continue assigning fronts
    k = 0
    while len(fronts[k]) > 0:
        Q = []
        for p in fronts[k]:
            for q in population[p]['dominates']:
                population[q]['dominated_by'].remove(p)
                if len(population[q]['dominated_by']) == 0:
                    Q.append(q)
        k += 1
        fronts.append(Q)

    return fronts

def crowding_distance(population, front, delivery_values, travel_times):
    distance = {i: 0 for i in front}
    for i in front:
        obj_1_i = objective_1(population[i]['visit'], delivery_values)
        obj_2_i = objective_2(population[i]['visit'], travel_times, len(delivery_values))
        
        # Calculate crowding distance here...
        # You can add your code to calculate the crowding distance based on objective values

    return distance

# Initialize Population
def create_population(pop_size, num_cities):
    population = []
    for _ in range(pop_size):
        visit = [randint(0, 1) for _ in range(num_cities)]  # Random solution
        population.append({
            'visit': visit
        })
    return population

# Crossover function (Two-point crossover)
def crossover(parent1, parent2):
    crossover_point = randint(0, len(parent1['visit']) - 1)
    offspring1 = parent1['visit'][:crossover_point] + parent2['visit'][crossover_point:]
    offspring2 = parent2['visit'][:crossover_point] + parent1['visit'][crossover_point:]
    return {'visit': offspring1}, {'visit': offspring2}

# Mutation function
def mutate(offspring, mutation_rate):
    for i in range(len(offspring['visit'])):
        if random() < mutation_rate:
            offspring['visit'][i] = 1 - offspring['visit'][i]  # Flip the bit (0->1 or 1->0)
    return offspring

# Selection function using tournament selection
def tournament_selection(population, fronts, k=2):
    front = fronts[0]  # First front (best solutions)
    selected = []
    for _ in range(k):
        ind1 = population[front[randint(0, len(front)-1)]]
        ind2 = population[front[randint(0, len(front)-1)]]
        selected.append(ind1 if random() < 0.5 else ind2)  # Simple selection (not optimal)
    return selected

# Main loop for NSGA-II (Iterate over generations)
def nsga_ii(pop_size, num_generations, delivery_values, travel_times, num_cities, mutation_rate=0.1):
    population = create_population(pop_size, num_cities)
    for gen in range(num_generations):
        fronts = non_dominated_sorting(population, delivery_values, travel_times)

        # Tournament Selection, Crossover, and Mutation
        new_population = []
        while len(new_population) < pop_size:
            parents = tournament_selection(population, fronts)
            offspring1, offspring2 = crossover(parents[0], parents[1])
            offspring1 = mutate(offspring1, mutation_rate)
            offspring2 = mutate(offspring2, mutation_rate)
            new_population.append(offspring1)
            new_population.append(offspring2)
        
        # Evaluate fitness (dominance sorting)
        fronts = non_dominated_sorting(new_population, delivery_values, travel_times)
        
        # Perform crowding distance calculation (for replacement and diversity)
        # Assuming we handle this in the sorting
        population = new_population  # Replace population with new one
    
    # After all generations, print the results
    fronts = non_dominated_sorting(population, delivery_values, travel_times)
    print("Pareto Front:")
    for i in fronts[0]:  # Get first front (non-dominated solutions)
        obj_1 = objective_1(population[i]['visit'], delivery_values)
        obj_2 = objective_2(population[i]['visit'], travel_times, num_cities)
        print(f"Solution {i} - Delivery Value: {obj_1}, Travel Time: {obj_2}, Visit: {population[i]['visit']}")

# Main Execution
if __name__ == "__main__":
    cities = ["A", "B", "C", "D", "E", "F", "G", "H"]
    num_cities = len(cities)
    delivery_values = [0, 500, 300, 400, 600, 200, 350, 450]
    time_windows = [(0, 24), (9, 12), (10, 14), (11, 13), (13, 17), (8.5, 10.5), (12, 16), (14, 18)]

    # Generate distance matrix and calculate travel times
    seed = 5286
    distance_matrix = generate_distances(num_cities, seed)
    speed = 30  # km/h
    travel_times = calculate_travel_times(distance_matrix, speed)

    # Run NSGA-II
    nsga_ii(pop_size=50, num_generations=100, delivery_values=delivery_values, 
            travel_times=travel_times, num_cities=num_cities)


Pareto Front:
Solution 1 - Delivery Value: 2800, Travel Time: 1175.7333333333327, Visit: [1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
import pandas as pd

# Load the Breast Cancer Dataset
file_path = 'Cancerdata.csv'
data = pd.read_csv(file_path)

data.head()
data.info()
print("Summary Statistics") 
data.describe()
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error, explained_variance_score


# Step 1: Define independent variables (X) and target variable (y)
X = data.iloc[:, :-1]  # All columns except the last (target)
y = data.iloc[:, -1]   # Target variable (last column)

# Step 2: Detect and remove outliers
# Using Z-score method for outlier detection
from scipy import stats
z_scores = np.abs(stats.zscore(X))  # Compute the Z-scores for X
threshold = 3  # Consider Z-scores greater than 3 as outliers
outlier_mask = (z_scores < threshold).all(axis=1)  # Keep rows where all Z-scores are below the threshold

# Apply the mask to remove outliers from the dataset
X_clean = X[outlier_mask]
y_clean = y[outlier_mask]

# Step 3: Split the data into training and testing sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X_clean, y_clean, test_size=0.3, random_state=42)

# Step 4: Standardize the features (important for linear regression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 5: Train the Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# Step 6: Make predictions on the test set
y_pred = lr_model.predict(X_test_scaled)

# Step 7: Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
medae = median_absolute_error(y_test, y_pred)
evs = explained_variance_score(y_test, y_pred)

# Print out the evaluation metrics
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R²): {r2}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Median Absolute Error (MedAE): {medae}")
print(f"Explained Variance Score (EVS): {evs}")

from statsmodels.stats.outliers_influence import variance_inflation_factor

# Compute VIF for each feature
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X_train_scaled, i) for i in range(X_train_scaled.shape[1])]

vif_data

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error, explained_variance_score
from scipy import stats


# Step 1: Define independent variables (X) and target variable (y)
X = data.iloc[:, :-1]  # All columns except the last (target)
y = data.iloc[:, -1]   # Target variable (last column)

# Step 2: Detect and remove outliers
# Using Z-score method for outlier detection
z_scores = np.abs(stats.zscore(X))  # Compute the Z-scores for X
threshold = 3  # Consider Z-scores greater than 3 as outliers
outlier_mask = (z_scores < threshold).all(axis=1)  # Keep rows where all Z-scores are below the threshold

# Apply the mask to remove outliers from the dataset
X_clean = X[outlier_mask]
y_clean = y[outlier_mask]

# Step 3: Split the data into training and testing sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X_clean, y_clean, test_size=0.3, random_state=42)

# Step 4: Standardize the features (important for linear regression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 5: Train the Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# Step 6: Make predictions on the test set
y_pred = lr_model.predict(X_test_scaled)

# Step 7: Evaluate the model using various metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
medae = median_absolute_error(y_test, y_pred)
evs = explained_variance_score(y_test, y_pred)

# Step 8: Print out the evaluation metrics
print("Model Evaluation Metrics:")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R²): {r2}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Median Absolute Error (MedAE): {medae}")
print(f"Explained Variance Score (EVS): {evs}")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error, explained_variance_score
from sklearn.decomposition import PCA
from scipy import stats
import statsmodels.api as sm

# Step 1: Define independent variables (X) and target variable (y)
X = data.iloc[:, :-1]  # All columns except the last (target)
y = data.iloc[:, -1]   # Target variable (last column)

# Step 2: Detect and remove outliers
# Using Z-score method for outlier detection
z_scores = np.abs(stats.zscore(X))  # Compute the Z-scores for X
threshold = 3  # Consider Z-scores greater than 3 as outliers
outlier_mask = (z_scores < threshold).all(axis=1)  # Keep rows where all Z-scores are below the threshold

# Apply the mask to remove outliers from the dataset
X_clean = X[outlier_mask]
y_clean = y[outlier_mask]

# Step 3: Split the data into training and testing sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X_clean, y_clean, test_size=0.3, random_state=42)

# Step 4: Standardize the features (important for PCA and linear regression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 5: Apply PCA for dimensionality reduction
pca = PCA(n_components=0.95)  # Choose the number of components to explain 95% of the variance
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Step 6: Train the Linear Regression model on the reduced dataset
lr_model = LinearRegression()
lr_model.fit(X_train_pca, y_train)

# Step 7: Make predictions on the test set
y_pred = lr_model.predict(X_test_pca)

# Step 8: Evaluate the model using various metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
medae = median_absolute_error(y_test, y_pred)
evs = explained_variance_score(y_test, y_pred)

# Step 9: Print out the evaluation metrics
print("Model Evaluation Metrics:")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R²): {r2}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Median Absolute Error (MedAE): {medae}")
print(f"Explained Variance Score (EVS): {evs}")

# Step 10: Check the significance of each independent variable using p-values
# Add constant to X_train_pca for intercept term (this is required for statsmodels)
X_train_with_intercept = sm.add_constant(X_train_pca)

# Fit the OLS (Ordinary Least Squares) regression model using statsmodels
ols_model = sm.OLS(y_train, X_train_with_intercept).fit()

# Print the summary of the OLS model to get p-values and other statistics
print("\nOLS Regression Summary:")
print(ols_model.summary())

# Step 11: Determine significance at α = 0.05
# Extract the p-values from the model summary
p_values = ols_model.pvalues[1:]  # Skip the intercept (first entry)

# Print the p-values and determine significance
for i, p_value in enumerate(p_values):
    feature = f"Principal Component {i+1}"  # Corresponding feature in PCA
    if p_value < 0.05:
        print(f"{feature} is significant (p-value: {p_value})")
    else:
        print(f"{feature} is not significant (p-value: {p_value})")


# Step 1: Fit the Logistic Regression model using sklearn (if not already done)
from sklearn.linear_model import LogisticRegression
import numpy as np


log_reg_model = LogisticRegression(solver='liblinear', penalty='l2')
log_reg_model.fit(X_train_scaled, y_train)

# Step 2: Extract the coefficients and calculate odds ratios
coefficients = log_reg_model.coef_[0]  # Coefficients for each feature
odds_ratios = np.exp(coefficients)  # Exponentiate to get the odds ratios

chosen_variables = X_train.columns[:5]

# Create a dictionary of the odds ratios for the chosen variables
odds_ratio_dict = dict(zip(chosen_variables, odds_ratios[:5]))

# Step 4: Display and interpret the odds ratios
print("\nOdds Ratios and Interpretation:")
for var, or_value in odds_ratio_dict.items():
    print(f"Odds ratio for {var}: {or_value:.2f}")
    
    if or_value > 1:
        print(f"Interpretation: A one-unit increase in {var} increases the odds of the outcome by a factor of {or_value:.2f}.")
    elif or_value < 1:
        print(f"Interpretation: A one-unit increase in {var} decreases the odds of the outcome by a factor of {1/or_value:.2f}.")
    else:
        print(f"Interpretation: {var} has no effect on the odds of the outcome.")


X = data[['mean radius', 'mean texture', 'mean smoothness', 'mean symmetry']]
y = data['target']  # Or whatever the target variable is

# Step 2: Split into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 3: Standardize the data (fit scaler on the training data only)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 4: Train the logistic regression model
from sklearn.linear_model import LogisticRegression
log_reg_model = LogisticRegression()
log_reg_model.fit(X_train_scaled, y_train)


input_features = np.array([[14.5, 18.0, 0.095, 0.180]])  # New patient's tumor features
input_features_scaled = scaler.transform(input_features)
predicted_prob = log_reg_model.predict_proba(input_features_scaled)[0, 1]  # Probability for class 1 (malignant)

# Step 6: Display prediction
print(f"Predicted Probability of Malignancy: {predicted_prob:.4f}")
if predicted_prob > 0.5:
    print("Prediction: Malignant")
else:
    print("Prediction: Benign")


# time series

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import numpy as np

# Step 1: Load the dataset
file_path = "time-series.csv"  # Path to the uploaded CSV file
data = pd.read_csv(file_path)

# Step 2: Extract the 'V6' column for analysis
time_series_cleaned = data['V6']

# Step 3: Plot the raw time series
plt.figure(figsize=(10, 6))
plt.plot(time_series_cleaned, marker='o', label='V6 (Sales)')
plt.title('Raw Time Series (V6)')
plt.xlabel('Time (Quarters)')
plt.ylabel('Sales')
plt.legend()
plt.grid(True)
plt.show()

# Step 4: Plot ACF and PACF
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Autocorrelation Function (ACF) plot
plot_acf(time_series_cleaned, lags=20, ax=axes[0])
axes[0].set_title('Autocorrelation (ACF)')

# Partial Autocorrelation Function (PACF) plot
plot_pacf(time_series_cleaned, lags=20, ax=axes[1])
axes[1].set_title('Partial Autocorrelation (PACF)')

plt.tight_layout()
plt.show()

# Step 5: Detect outliers using Z-scores
z_scores = np.abs((time_series_cleaned - time_series_cleaned.mean()) / time_series_cleaned.std())
outliers = time_series_cleaned[z_scores > 3]  # Threshold of 3 for detecting outliers

# Print outliers
if not outliers.empty:
    print("Outliers detected:")
    print(outliers)
else:
    print("No outliers detected.")

# Replace outliers with the median value
median_value = time_series.median()
time_series_cleaned = time_series.copy()
time_series_cleaned[z_scores > 3] = median_value

# Plot the cleaned time series
plt.figure(figsize=(10, 6))
plt.plot(time_series_cleaned, label='Cleaned V6 (Sales)', marker='o')
plt.title('Cleaned Time Series (V6)')
plt.xlabel('Time (Quarters)')
plt.ylabel('Sales')
plt.legend()
plt.show()

from statsmodels.tsa.stattools import adfuller

# Perform the Augmented Dickey-Fuller (ADF) test
adf_result = adfuller(time_series_cleaned)

# Display the ADF statistic and p-value
print(f"ADF Statistic: {adf_result[0]:.4f}")
print(f"p-value: {adf_result[1]:.4f}")

# Interpret the test results
if adf_result[1] < 0.05:
    print("The time series is stationary.")
else:
    print("The time series is not stationary; further transformations are required.")

from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore")  
arima_model = ARIMA(time_series_cleaned, order=(1, 1, 1))  # Example order
arima_result = arima_model.fit()


print(arima_result.summary())
from statsmodels.stats.diagnostic import het_breuschpagan, acorr_ljungbox
from statsmodels.graphics.tsaplots import plot_acf
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt

# Step 1: Plot the residuals
residuals = arima_result.resid

plt.figure(figsize=(10, 6))
plt.plot(residuals, marker='o', label='Residuals', color='blue')
plt.axhline(0, linestyle='--', color='red')
plt.title('Residuals of ARIMA Model', color='darkgreen')
plt.xlabel('Time (Quarters)', color='darkblue')
plt.ylabel('Residuals', color='darkblue')
plt.legend()
plt.grid(True)
plt.show()

# Step 2: Plot ACF of residuals
plt.figure(figsize=(10, 6))
plot_acf(residuals, lags=20, color='purple')
plt.title('ACF of Residuals', color='darkgreen')
plt.grid(True)
plt.show()

# Step 3: Histogram of residuals to check normality
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True, color='orange')
plt.title('Histogram of Residuals', color='darkgreen')
plt.xlabel('Residual Values', color='darkblue')
plt.ylabel('Frequency', color='darkblue')
plt.grid(True)
plt.show()

# Step 4: Perform Ljung-Box test for autocorrelation
ljung_box_test = acorr_ljungbox(residuals, lags=20)
print("Ljung-Box Test Results:")
print(ljung_box_test)

# Step 5: Perform Jarque-Bera test for normality
jb_test = stats.jarque_bera(residuals)
print("Jarque-Bera Test Results:")
print(f"JB Statistic: {jb_test[0]:.4f}, p-value: {jb_test[1]:.4f}")

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore


# Calculate Z-scores for the 'V6' column
z_scores = zscore(data['V6'])

# Identify outliers based on Z-score threshold (e.g., abs(z) > 3)
outliers = np.where(np.abs(z_scores) > 3)

# Print the outlier indices and values
print(f"Outliers detected at indices: {outliers[0]}")
print(f"Outlier values: {data.iloc[outliers[0]]}")

# Remove outliers by replacing them with NaN
data_cleaned = data.copy()
data_cleaned.iloc[outliers[0], data_cleaned.columns.get_loc('V6')] = np.nan

# Replace NaN values in the 'V6' column with the median
data_cleaned['V6'].fillna(data_cleaned['V6'].median(), inplace=True)

# Plot the original and cleaned data for comparison
plt.figure(figsize=(10, 6))
plt.plot(data['V6'], label='Original Data', color='blue', alpha=0.6)
plt.plot(data_cleaned['V6'], label='Data after Outlier Removal', color='red', alpha=0.6)
plt.title('Time Series with Outliers Removed')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()

import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

# Ensure data_cleaned is numeric and handle missing values
data_cleaned = pd.to_numeric(data_cleaned, errors='coerce')  # Convert to numeric, coercing errors
data_cleaned = data_cleaned.dropna()  # Drop NaNs (or use fillna() if needed)

# Train-test split for model evaluation (80% train, 20% test)
train_size = int(len(data_cleaned) * 0.8)
train, test = data_cleaned[:train_size], data_cleaned[train_size:]

# Initialize containers for evaluation metrics
models = ['ARIMA', 'SARIMA']
aic_values = []
bic_values = []
mse_values = []
r2_values = []
model_predictions = {}

# --- ARIMA Model ---
print("Fitting ARIMA Model...")
arima_model = ARIMA(train, order=(1, 1, 1))  # Example order (p=1, d=1, q=1)
arima_result = arima_model.fit()

# Forecasting the test set
arima_forecast = arima_result.forecast(steps=len(test))

# Evaluate ARIMA
arima_mse = mean_squared_error(test, arima_forecast)
arima_r2 = r2_score(test, arima_forecast)
aic_values.append(arima_result.aic)
bic_values.append(arima_result.bic)
mse_values.append(arima_mse)
r2_values.append(arima_r2)
model_predictions['ARIMA'] = arima_forecast

# --- SARIMA Model ---
print("Fitting SARIMA Model...")
sarima_model = SARIMAX(train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 4))  # Example SARIMA model
sarima_result = sarima_model.fit()

# Forecasting the test set
sarima_forecast = sarima_result.forecast(steps=len(test))

# Evaluate SARIMA
sarima_mse = mean_squared_error(test, sarima_forecast)
sarima_r2 = r2_score(test, sarima_forecast)
aic_values.append(sarima_result.aic)
bic_values.append(sarima_result.bic)
mse_values.append(sarima_mse)
r2_values.append(sarima_r2)
model_predictions['SARIMA'] = sarima_forecast

# --- Additional Model (Optional: Example with ARMA or other variations) ---
# You can add other models here, like ARMA, Exponential Smoothing, etc.
# For simplicity, we skip this step in this example

# --- Compare Models ---
print("\nModel Comparison:")

# Display model evaluation metrics (AIC, BIC, MSE, R^2)
model_comparison = pd.DataFrame({
    'Model': models,
    'AIC': aic_values,
    'BIC': bic_values,
    'MSE': mse_values,
    'R^2': r2_values
})

print(model_comparison)

# --- Residual Analysis ---
# ARIMA Residuals
arima_residuals = arima_result.resid

# SARIMA Residuals
sarima_residuals = sarima_result.resid

# Plotting the actual vs predicted values for both models
plt.figure(figsize=(12, 6))

# Plot Actual vs Predicted for ARIMA
plt.subplot(1, 2, 1)
plt.plot(test.index, test, label='Actual', color='blue')
plt.plot(test.index, arima_forecast, label='ARIMA Predicted', color='red')
plt.title('ARIMA Model: Actual vs Predicted')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()

# Plot Actual vs Predicted for SARIMA
plt.subplot(1, 2, 2)
plt.plot(test.index, test, label='Actual', color='blue')
plt.plot(test.index, sarima_forecast, label='SARIMA Predicted', color='green')
plt.title('SARIMA Model: Actual vs Predicted')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()

plt.tight_layout()
plt.show()

# Residual Analysis
plt.figure(figsize=(12, 6))

# Plot ARIMA residuals
plt.subplot(1, 2, 1)
plt.plot(arima_residuals)
plt.title('ARIMA Residuals')
plt.axhline(y=0, color='black', linestyle='--')

# Plot SARIMA residuals
plt.subplot(1, 2, 2)
plt.plot(sarima_residuals)
plt.title('SARIMA Residuals')
plt.axhline(y=0, color='black', linestyle='--')

plt.tight_layout()
plt.show()

# --- Final Conclusion ---
# Based on the AIC, BIC, MSE, and R² values, the best model can be selected
best_model = model_comparison.loc[model_comparison['MSE'].idxmin()]['Model']
print(f"\nBest Model: {best_model}")

# --- Best Model Selection ---
best_model_name = model_comparison.loc[model_comparison['MSE'].idxmin()]['Model']
print(f"Best Model: {best_model_name}")

# --- Forecasting for the Next Four Quarters ---
forecast_steps = 4  # Forecasting for the next four quarters

# ARIMA Forecasting
if best_model_name == 'ARIMA':
    print("Using ARIMA for forecasting...")
    arima_forecast = arima_result.forecast(steps=forecast_steps)

    # Plotting the forecasted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, label='Actual', color='blue')
    plt.plot(pd.date_range(test.index[-1], periods=forecast_steps, freq='Q'), arima_forecast, label='ARIMA Forecast', color='red')
    plt.title('ARIMA Model Forecast for Next Four Quarters')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.legend()
    plt.show()

# SARIMA Forecasting
elif best_model_name == 'SARIMA':
    print("Using SARIMA for forecasting...")
    sarima_forecast = sarima_result.forecast(steps=forecast_steps)

    # Plotting the forecasted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, label='Actual', color='blue')
    plt.plot(pd.date_range(test.index[-1], periods=forecast_steps, freq='Q'), sarima_forecast, label='SARIMA Forecast', color='green')
    plt.title('SARIMA Model Forecast for Next Four Quarters')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.legend()
    plt.show()

